In [ ]:
from vectice import connect, S3Resource, Dataset, Model
from boto3 import client 
from botocore import UNSIGNED
from botocore.client import Config

s3_client = client('s3', config=Config(signature_version=UNSIGNED), region_name='us-west-1')
users_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/users.csv")
comp_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/competitor_info.csv")
clean_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/competitor_info.csv")
train_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/traindataset.csv")
test_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/testdataset.csv")
val_ds = S3Resource(s3_client, bucket_name='vectice-examples' ,resource_path="Tutorial/ForecastTutorial/validatedataset.csv")

In [ ]:
vtc_prj = connect(config="vectice-config.json")

In [ ]:
# List out the phases in the project
vtc_prj.list_phases()

In [ ]:
vtc_phase = vtc_prj.phase(34412)

In [ ]:
vtc_phase.list_step_definitions()

In [ ]:
# Document the project origin datasets
vtc_iter = vtc_phase.iteration()
vtc_iter.step_data_collection = Dataset.origin( name="Users", resource=users_ds)
vtc_iter.step_data_collection += Dataset.origin( name="Competitor_Info", resource=comp_ds)

In [ ]:
# EDA on the datasets
...

# Document our EDA process
vtc_iter.step_eda = Dataset.clean(name="Clean Dataset",resource=clean_ds, derived_from=[23974, 23975])


# Split datasets for model and register them
...

vtc_iter.step_split_dataset = Dataset.modeling(name="Modeling Dataset", training_resource=train_ds,testing_resource=test_ds,validation_resource=val_ds)


# All steps on the phase are completed, mark the iteration as complete
vtc_iter.complete()

In [ ]:
# Get the next phase (modeling)
vtc_iter = vtc_prj.phase(34413).iteration()

# Build the model
...

# Document the model technique used
vtc_iter.step_document_modeling_technique = "Using a Linear Regression model."

# Connecting your model to an iteration will automatically sync the information to Vectice UI
vtc_iter.step_build_model = Model(library="sklearn", 
                                      technique="ln", 
                                      attachments=["Feature_Importance.png", "26_LiftChart_holdout.png"], 
                                      name="Base_LR_Model")


# Fine tune hyper-parameters
...

# Register model in Vectice
vtc_iter.step_fine_tune_model = Model(library="sklearn", 
                                      technique="ln", 
                                      attachments=["Feature_Importance.png", "26_LiftChart_holdout.png"], 
                                      name="Tuned_LR_Model",
                                      metrics={"metric_1": 1, "metric_2": 1}, 
                                      properties={"prop": "value"})

# All steps on the phase are completed, mark the iteration as complete
vtc_iter.complete()